In [1]:
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50

torch.manual_seed(42)

In [2]:
# Device Configure
device = torch.device('cpu') # m1 CPU

In [3]:
batch_size = 512
num_workers = 4

In [4]:
# Dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='../../data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=num_workers)

testset = torchvision.datasets.CIFAR10(root='../../data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=num_workers)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# Model
net = resnet50(pretrained=False).to(device)

In [6]:
# Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [7]:
def train_engine(epochs=10):
    net.train()
    
    total_time = time.time()
    for epoch in range(epochs):  # loop over the dataset multiple times
        
        correct, total = 0, 0
        running_loss = 0.0
        start_time = time.time()
        
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device)
        
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # statistics
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            running_loss += loss.item()
        
        print(f'Epoch {epoch+1}: takes {(time.time() - start_time):.2f} seconds, loss: {(running_loss / (i+1)):.3f}, acc: {(correct/total*100):.2f}%')
    
    total_time = time.time() - total_time
    print(f'Finished training in {total_time:.2f} seconds; Avg: {(total_time/epochs):.2f} seconds')
    
    net.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    print(f'Test accuracy:{(correct/total*100):.2f}%')

In [8]:
# train
train_engine(epochs = 20)

Epoch 1: takes 233.25 seconds, loss: 2.634, acc: 13.48%
Epoch 2: takes 237.92 seconds, loss: 2.138, acc: 21.25%
Epoch 3: takes 240.12 seconds, loss: 1.976, acc: 27.28%
Epoch 4: takes 231.39 seconds, loss: 1.853, acc: 31.81%
Epoch 5: takes 230.86 seconds, loss: 1.750, acc: 35.96%
Epoch 6: takes 231.92 seconds, loss: 1.664, acc: 39.02%
Epoch 7: takes 230.84 seconds, loss: 1.584, acc: 42.22%
Epoch 8: takes 230.72 seconds, loss: 1.507, acc: 45.26%
Epoch 9: takes 232.54 seconds, loss: 1.437, acc: 48.08%
Epoch 10: takes 232.06 seconds, loss: 1.360, acc: 51.10%
Epoch 11: takes 231.42 seconds, loss: 1.286, acc: 53.82%
Epoch 12: takes 232.25 seconds, loss: 1.211, acc: 56.48%
Epoch 13: takes 232.97 seconds, loss: 1.139, acc: 59.46%
Epoch 14: takes 232.01 seconds, loss: 1.059, acc: 62.42%
Epoch 15: takes 232.37 seconds, loss: 0.979, acc: 65.19%
Epoch 16: takes 230.21 seconds, loss: 0.889, acc: 68.84%
Epoch 17: takes 231.74 seconds, loss: 0.827, acc: 70.89%
Epoch 18: takes 240.29 seconds, loss: 0.